In [ ]:
"""
Generic Paginated API Scraper
--------------------------------
This script fetches paginated data from a REST API endpoint
and progressively saves results to a CSV file.

Designed for:
- Large datasets
- Crash recovery (resume logic)
- Safe incremental saving

Replace the placeholders below with your own API details.
"""

import requests
import pandas as pd
import time
import os
import sys


# ==========================================================
# CONFIGURATION
# ==========================================================

# 🔹 Replace with your own API endpoint
BASE_URL = "https://example.com/api/endpoint"  # <-- ADD YOUR API URL HERE

# 🔹 Total number of pages available in the API
TOTAL_PAGES = 1000  # <-- UPDATE THIS VALUE

# 🔹 Output CSV filename
SAVE_FILE = "output_data.csv"

# 🔹 Custom headers (modify if needed)
headers = {
    "User-Agent": "Mozilla/5.0"  # <-- Customize if required
}


# ==========================================================
# RESUME LOGIC (Optional but recommended)
# ==========================================================

start_page = 1

if os.path.exists(SAVE_FILE):
    existing_df = pd.read_csv(SAVE_FILE)
    rows_existing = len(existing_df)

    # ⚠️ Assumes fixed number of records per page
    # Adjust divisor if your API returns a different page size
    records_per_page = 10  # <-- UPDATE IF DIFFERENT
    pages_done = rows_existing // records_per_page

    start_page = pages_done + 1
    print(f"Resuming from page {start_page}")


# ==========================================================
# MAIN LOOP
# ==========================================================

for page in range(start_page, TOTAL_PAGES + 1):
    try:
        # API query parameters
        params = {
            "page": page  # <-- MODIFY PARAMS IF NEEDED
        }

        response = requests.get(
            BASE_URL,
            params=params,
            headers=headers,
            timeout=15  # Prevents hanging requests
        )

        response.raise_for_status()

        # Parse JSON response
        data = response.json()

        rows = []

        # 🔹 Modify field names according to your API structure
        for item in data["data"]:  # <-- Adjust JSON path if needed
            rows.append({
                "Field_1": item.get("field_1", ""),  # <-- CHANGE
                "Field_2": item.get("field_2", ""),  # <-- CHANGE
                "Field_3": item.get("field_3", ""),  # <-- CHANGE
            })

        df_page = pd.DataFrame(rows)

        # Append to CSV safely
        df_page.to_csv(
            SAVE_FILE,
            mode='a',
            header=not os.path.exists(SAVE_FILE),
            index=False
        )

        # Single-line progress display
        sys.stdout.write(f"\rPage {page} / {TOTAL_PAGES}")
        sys.stdout.flush()

        # Optional polite delay
        time.sleep(0.2)

    except Exception as e:
        print(f"\nError on page {page}: {e}")
        time.sleep(3)


print("\nDONE ✅")



Error on page 1: HTTPSConnectionPool(host='example.com', port=443): Max retries exceeded with url: /api/endpoint?page=1 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1032)')))

Error on page 2: HTTPSConnectionPool(host='example.com', port=443): Max retries exceeded with url: /api/endpoint?page=2 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1032)')))

Error on page 3: HTTPSConnectionPool(host='example.com', port=443): Max retries exceeded with url: /api/endpoint?page=3 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1032)')))
